In [1]:
import pdb
import json
import glob
import pickle
import tokenize
from io import StringIO
import numpy as np
import sys
from collections import Counter

In [ ]:
with open('pickles/matplotlib_dataset.pkl', 'rb') as fp:
    matplotlib_data = pickle.load(fp)

In [2]:
with open('/home/anushap/Code-Generation-Old/pandas_context_dataset_5years_no_dups.pkl', 'rb') as fp:
    pandas_data = pickle.load(fp)

In [3]:
len(pandas_data)

33205

In [ ]:
len(matplotlib_data)

In [ ]:
code = [''.join(item[0]) for item in matplotlib_data]
intent = [''.join(item[1]) for item in matplotlib_data]

# Filtering \n s from code and intent

In [ ]:
line_stripped_data = []
for item in pandas_data:
    code, intent = [], []
    for line in item[0]:
        if line == '\n':
            continue
        code.append(line)
    for text in item[1]:
        if text == '\n':
            continue
        intent.append(text)
    line_stripped_data.append((code, intent))

In [ ]:
len(line_stripped_data)

# Filtering non ascii characters from code and intent

In [ ]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

In [ ]:
ascii_filtered_data = []
for item in line_stripped_data:
    code, intent = [], []
    for line in item[0]:
        if is_ascii(line):
            code.append(line)
    
    for line in item[1]:
        if is_ascii(line):
            intent.append(line)
            
    if code == [] or intent == []:
        continue
    ascii_filtered_data.append((code, intent))

In [ ]:
len(ascii_filtered_data)

# Filtering examples which are only 3 lines (both code and intent)

In [ ]:
lines_reduced_data = []
for item in ascii_filtered_data:
    if 0 < len(item[0]) <= 3 and 0 < len(item[1]) <= 3:
        lines_reduced_data.append(item)

In [ ]:
len(lines_reduced_data)

# Filtering based on Pandas functions

In [ ]:
with open('pickles/pandas_fns_names.pkl', 'rb') as fp:
    pandas_fns = pickle.load(fp)

In [ ]:
len(pandas_fns)

In [ ]:
def contains_pandas_api(codeline):
    for func in pandas_fns:
        if '.'+func in codeline:
            return True
    return False

In [ ]:
pandas_fn_filtered_data = []
for item in lines_reduced_data:
    code = item[0]
    if any(list(map(contains_pandas_api, code))):
        pandas_fn_filtered_data.append(item)

In [ ]:
len(pandas_fn_filtered_data)

In [ ]:
pandas_fn_filtered_data[3]

# Filtering based on pyplot functions

In [ ]:
fp = open('pyplot_fns.txt', 'r')
fp.readline()
fn_lines = fp.readlines()
fp.close()

In [ ]:
pyplot_functions = []
for line in fn_lines:
    arr = line.split()
    pyplot_functions.append(arr[0])

In [ ]:
len(pyplot_functions)

In [ ]:
def contains_pyplot_api(codeline):
    for func in pyplot_functions:
        if '.'+func in codeline:
            return True
    return False

In [ ]:
pyplot_data = []
for item in matplotlib_data:
    code = item[0]
    if any(list(map(contains_pyplot_api, code))):
        pyplot_data.append(item)

In [ ]:
len(pyplot_data)

In [ ]:
pyplot_data[20]

# For Pandas 

In [ ]:
fp = open('pandas_fn_raw.txt', 'r')
data = fp.readlines()
fp.close()

In [ ]:
pandas_fns = []

for line in data:
    if "(" in line:
        arr = line.split("(")
        if '.' in arr[0]:
            new_arr = arr[0].split(".")
            if len(new_arr) > 1:
                new_arr = new_arr[1].split(" ")
                pandas_fns.append(new_arr[0].replace('\n', ''))
            else:
                pandas_fns.append(new_arr[1].replace('\n', ''))
        else:
            pandas_fns.append(arr[0].replace('\n', ''))
    else:
        arr = line.split('.')
        if len(arr)>1:
            new_arr = arr[1].split(" ")
            pandas_fns.append(new_arr[0].replace('\n', ''))
        else:
            if len(arr) > 1:
                pandas_fns.append(arr[1].replace('\n', ''))


In [ ]:
len(pandas_fns)

In [ ]:
pandas_fns = set(pandas_fns)

In [ ]:
len(pandas_fns)

In [ ]:
with open('pickles/pandas_fns_names.pkl', 'wb') as fp:
    pickle.dump(pandas_fns, fp)

In [ ]:
len(pandas_data)